This MMS case verifies the implementation of the Surface Kinetics boundary condition in FESTIM. We will consider a time-dependent case of hydrogen diffusion in a unit interval domain $\Omega$ with a homogeneous diffusion coefficient $D$, and a Dirichlet boundary condition on the rear domain side.

The diffusion problem is:
$$ 
\begin{align}
    &\dfrac{\partial c_\mathrm{m}}{\partial t} = \nabla\cdot\left(D\nabla c_\mathrm{m} \right) + S \quad \textrm{ on } \Omega, \\
    &-D \nabla c_\mathrm{m} \cdot \mathbf{n} = \lambda_{\mathrm{IS}} \dfrac{\partial c_{\mathrm{m}}}{\partial t} + J_{\mathrm{bs}} - J_{\mathrm{sb}} \quad \textrm{ at } x=0, \\
    &c_\mathrm{m} = c_\mathrm{0,m} \quad \textrm{ at } x=1, \\
    &c_\mathrm{m} = c_\mathrm{0,m} \quad \textrm{ at } t=0, \\
\end{align}
$$

Evolution of surface concentration is governed by:
$$ 
\begin{align}
    &\dfrac{d c_\mathrm{s}}{d t} = J_{\mathrm{bs}} - J_{\mathrm{sb}} + J_{\mathrm{vs}}  \quad \textrm{ at } x=0 \\
    &c_\mathrm{s}= c_\mathrm{0,s}\quad \textrm{ at } t=0, \\
\end{align}
$$